# 示例：使用OSPFv3的Starlink卫星网络

## 概述
本示例构建两个个卫星网络，分别为20 * 20 Grid拓扑和10 * 10 Grid拓扑。
每个卫星搭在一个星载路由器，运行OSPFv3协议。随后在卫星和地面站中宣告路由，观察路由传播情况，并使用ping观察星间、星地传输路径的时延。
本示例中，运行两个示例实验：
1. 前端界面注入异常实验：通过前端界面使一条链路发生故障，观察网络路由收敛情况；
2. 脚本注入异常实验：通过脚本使一条链路发生故障，观察网络路由收敛情况；

### 卫星配置
+ 接口设置：共设置5个网络接口eth0-eth4。其中eth0-eth3是星间链路接口，对端目标是邻居卫星（具体参考[此文档](../../satnet/satnet_phy_engine.md#接口配置)）；eth4是星地链路接口，对端目标是信关站。
+ ip设置：设置lo地址为`fd01::x:y:1`，其中x为该卫星的轨道编号，y为该卫星在轨内的编号。
+ ospf设置：开启eth0-eth4的ospfv3功能，所有接口的area均设为backbone area(0.0.0.0)，即把所有星间、星地链路囊括在backbone area中。将ospf路由器id设置为`192.x.y.1`其中x为该卫星的轨道编号，y为该卫星在轨内的编号。通过`redistribute connected`将本卫星已经连接的路由（此示例中为通向本机lo地址的路由）通过ospf宣告给邻居。

### 信关站配置
+ 接口设置：设置1个星地链路接口eth0，即只有一根天线。
+ ip设置：设置lo地址为`fd02::x:1/128`，其中x为该信关站的编号。
+ ospf设置：开启eth0的ospfv3功能，area均设为backbone area(0.0.0.0)。将ospf路由器id设置为`172.16.x.1`其中x为该信关站的编号。通过`redistribute connected`将本信关站已经连接的路由（此示例中为通向本机lo地址的路由）通过ospf宣告给邻居。

## 操作步骤

In [2]:
API_KEY = '6be9eb101c'
BASE_URL = f'https://{API_KEY}.backend.sernes.cn'
# 网络配置
NETWORK_NAME = 'satnet_ospf_100'

from tools import APIClient, WebSocketClient
import asyncio
import time

client = APIClient(BASE_URL)

### 步骤一：创建网络
在后台管理界面中，应用该网络，进入可视化界面。



In [8]:
# 创建网络的请求参数
network_params = {
  "network_name": NETWORK_NAME,
  "network_description": "Satellite Network with OSPFv3",
  "constellation": {
    "name": "my_constellation",
    "type": "walker_delta",
    "orbit_altitude": 570,
    "orbit_inclination": 70,
    "orbit_num": 10,
    "sat_num_per_orbit": 10,
    "phase_shift": 1,
    "sat_isl_link_num": 4,
    "sat_gsl_link_num": 1,
    "sat_access_link_num": 0
  },
  "gs_set": [
    {
      "name": "gs_0",
      "latitude": 37.77,
      "longitude": -122.42,
      "elevation": 121.1,
      "gs_antenna_num": 1,
      "gs_antenna_angle": 25,
    },
    {
      "name": "gs_1",
      "latitude": 38.91,
      "longitude": -77.01,
      "elevation": 126.1,
      "gs_antenna_num": 1,
      "gs_antenna_angle": 25,
    }
  ],
}
# 发送创建网络的POST请求
print(network_params)
response = client.post('/api/network/create/', json_data=network_params)
if response:
    print("网络创建成功")
    network_id = response['network_id']
else:
    print("网络创建失败")

{'network_name': 'satnet_ospf_100', 'network_description': 'Satellite Network with OSPFv3', 'constellation': {'name': 'my_constellation', 'type': 'walker_delta', 'orbit_altitude': 570, 'orbit_inclination': 70, 'orbit_num': 10, 'sat_num_per_orbit': 10, 'phase_shift': 1, 'sat_isl_link_num': 4, 'sat_gsl_link_num': 1, 'sat_access_link_num': 0}, 'gs_set': [{'name': 'gs_0', 'latitude': 37.77, 'longitude': -122.42, 'elevation': 121.1, 'gs_antenna_num': 1, 'gs_antenna_angle': 25}, {'name': 'gs_1', 'latitude': 38.91, 'longitude': -77.01, 'elevation': 126.1, 'gs_antenna_num': 1, 'gs_antenna_angle': 25}]}
网络创建成功


In [9]:
client.post(f'/api/network/{network_id}/file/upload', files={'file': ('config.zip', open(f'{NETWORK_NAME}.zip', 'rb'))})

{'status': 'success',
 'path': '/mnt/reals-driver/nfs/netconf/network3/config_network3_20250508_155140.zip'}

### 步骤二：应用网络
在后台管理界面中，应用该网络，进入可视化界面。



In [5]:
# 发送启动网络的POST请求
client.get(f'/api/network/{network_id}/run/')
# 检查网络是否启动成功
while True:
    response = client.get(f'/api/network/list', {
        'id': network_id
    })
    network_status = response['data'][0]['network_status']
    if network_status == 1:
        print('网络启动成功')
        break
    time.sleep(1)

网络启动成功


### 步骤三：创建场景
在可视化界面中，点击“创建场景”，场景参数配置如下：

- 场景时间：按需设置（推荐300s以上）
- 异轨星间链路异常发生概率：0
- 日凌判定链路太阳光夹角：0

其他参数保持默认即可。创建场景后，等待该场景计算完成，点击播放按钮，运行该场景。

### 步骤四：验证星间路由连通性
点击任一卫星，进入其终端（以Sat90为例），输入如下命令查看路由：

在Sat90上，向Sat10发送ping请求：
```
Sat90# ping fd01::1:0:1
PING fd01::16:1:1 (fd01::16:1:1): 56 data bytes
64 bytes from fd01::16:1:1: seq=0 ttl=53 time=82.606 ms
64 bytes from fd01::16:1:1: seq=1 ttl=53 time=82.065 ms
64 bytes from fd01::16:1:1: seq=2 ttl=53 time=82.716 ms
64 bytes from fd01::16:1:1: seq=3 ttl=53 time=82.127 ms
64 bytes from fd01::16:1:1: seq=4 ttl=53 time=82.614 ms
64 bytes from fd01::16:1:1: seq=5 ttl=53 time=82.363 ms
```
发现可以ping通。

### 步骤五：脚本注入异常实验
停止场景，重新运行该场景。在Sat90中执行`ping fd01::1:0:1`向Sat321发送ping请求。

在场景运行过程中，通过脚本[isl_batch_operation.py](isl_batch_operation.py)向卫星网络注入链路异常。在任意环境中（如用户自己的PC）执行：

In [5]:
!python3 isl_batch_operation.py down -r 0.2 --api-key $API_KEY

Link set operation succeeded.


In [6]:
!python3 isl_batch_operation.py up --api-key $API_KEY

Link set operation succeeded.


上述脚本执行后，网络中20%的链路被设置为down（不可用）。可以在前端看见橙色链路为被设置为down的链路

注入异常后，卫星网络需要通过ospf重新计算路由，因此通信会中断一段时间。观察Sat123向Sat321的ping情况，发现在seq=50后约42s的时间间隔内通信中断，证明ospf重新收敛过程中，Sat123至Sat321的路由不可用。随后在seq=92时通信重新建立，证明ospf重新收敛。然而由于收敛后的路径比源路径更长（大量链路不可用导致需要新路径需要绕路），ping的时延相较于异常注入前有所增加。

```
64 bytes from fd01::16:1:1: seq=46 ttl=53 time=95.167 ms
64 bytes from fd01::16:1:1: seq=47 ttl=53 time=94.949 ms
64 bytes from fd01::16:1:1: seq=48 ttl=53 time=95.522 ms
64 bytes from fd01::16:1:1: seq=49 ttl=53 time=94.697 ms
64 bytes from fd01::16:1:1: seq=50 ttl=53 time=94.079 ms
64 bytes from fd01::16:1:1: seq=92 ttl=49 time=179.849 ms #异常！
64 bytes from fd01::16:1:1: seq=94 ttl=49 time=131.151 ms
64 bytes from fd01::16:1:1: seq=95 ttl=49 time=156.219 ms
64 bytes from fd01::16:1:1: seq=96 ttl=51 time=188.035 ms
64 bytes from fd01::16:1:1: seq=97 ttl=49 time=151.281 ms
64 bytes from fd01::16:1:1: seq=98 ttl=49 time=143.217 ms
```